# Univariate local join counts (LJC), bivariate LJC, multivariate LJC

### Univariate local join counts (LJC)


To review, the global black-black (1-1) count is the total across the entire study area:

$$BB = \sum_{i} \sum_j w_{ij} x_{i} x_{j}$$

However, the local is: 

$$BB_i = x_i \sum_{j} w_{ij} x_j$$

...where a count of the neighbors with an observation of $x_j=1$ for those locations where $x_i=1$. This focuses on the BB counts of a given polygon (x_i).

What we will do now is to remake the data as it appears in the docstrings of the join_count function and try to get the bivariate, then multivariate, up and running.



In [100]:
import numpy as np
import libpysal
import pandas as pd

# Create a 16x16 grid
w = libpysal.weights.lat2W(4, 4)
y_1 = np.ones(16)
# Set the first 9 of the ones to 0
y_1[0:8] = 0
print('new y_1', y_1)

new y_1 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]


Carry out some standardization processes

In [101]:
# Flatten the input vector y
y_1 = np.asarray(y_1).flatten()
print(y_1)
# ensure weights are binary transformed
w.transformation = 'b'

[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]


Identify univariate local join counts

In [102]:
adj_list = w.to_adjlist(remove_symmetric=False) 
print(adj_list)
zseries_var1 = pd.Series(y_1, index=w.id_order)
focal_var1 = zseries_var1.loc[adj_list.focal].values
neighbor_var1 = zseries_var1.loc[adj_list.neighbor].values
# Identify which adjacency lists are both equal to 1
BBs = (focal_var1 == 1) & (neighbor_var1 == 1)
BBs
# also convert to a 0/1 array
BBs.astype('uint8')

    focal  neighbor  weight
0       0         4     1.0
1       0         1     1.0
2       1         0     1.0
3       1         5     1.0
4       1         2     1.0
5       2         1     1.0
6       2         6     1.0
7       2         3     1.0
8       3         2     1.0
9       3         7     1.0
10      4         0     1.0
11      4         8     1.0
12      4         5     1.0
13      5         1     1.0
14      5         4     1.0
15      5         9     1.0
16      5         6     1.0
17      6         2     1.0
18      6         5     1.0
19      6        10     1.0
20      6         7     1.0
21      7         3     1.0
22      7         6     1.0
23      7        11     1.0
24      8         4     1.0
25      8        12     1.0
26      8         9     1.0
27      9         5     1.0
28      9         8     1.0
29      9        13     1.0
30      9        10     1.0
31     10         6     1.0
32     10         9     1.0
33     10        14     1.0
34     10        11 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=uint8)

Now we need to map these T/F to the arrangement of the vector. While I got the correct output, it's a bit messy. This part is tricky and will likely need optimization - can we exploit the natural structure of the adjacency list without needing to make a new dataframe and run a groupby? 

In [103]:
# Create a df that uses the adjacency list focal values and the BBs counts
temp = pd.DataFrame(adj_list.focal.values, BBs.astype('uint8')).reset_index()
# Temporarily rename the columns
temp.columns = ['BB', 'ID']
temp = temp.groupby(by='ID').sum()
temp.BB.values

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 3, 2, 2, 3, 3, 2], dtype=uint64)

We will still need to sort out inference!

## Bivariate local join counts

Moving on to bivariate local join counts. Using the nomenclature `x` and `z` to represent the two variables.

Firstly, the **co-location** of the events at location $i$ need to be taken into account. As explained in the other workbook, there are two kinds of bivariate local join counts considered. 

**Case 1: no in-situ co-location**: 'where $x_i$ and $z_i$ do NOT take on the same value at either location $i$ (itself) or $j$ (neighbors)

- Example: when $x_i=1$ for location $i$, then $z_i=0$. We count the number of neighbors of $i$ when $x_i=1$ for which the value of $z_j=1$ but the value of $x_j=0$

This effectively becomes a combination of identifcation where `x` is in a black(1)-white(0) join, and where `y` is in a white(0)-black(1) join.

Let's experiment...

In [97]:
x = y_1
z = [0,1,0,1,1,1,1,1,0,0,1,1,0,0,1,1]

print('x', x)
print('z', z)

x [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
z [0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1]


Carry out some standardization procedures

In [98]:
# Flatten the input vector y
x = np.asarray(x).flatten()
z = np.asarray(z).flatten()
print(x)
print(z)
# ensure weights are binary transformed
w.transformation = 'b'

[0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
[0 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1]


Create adjacency list

In [99]:
adj_list = w.to_adjlist(remove_symmetric=False) 
print(adj_list)

    focal  neighbor  weight
0       0         4     1.0
1       0         1     1.0
2       1         0     1.0
3       1         5     1.0
4       1         2     1.0
5       2         1     1.0
6       2         6     1.0
7       2         3     1.0
8       3         2     1.0
9       3         7     1.0
10      4         0     1.0
11      4         8     1.0
12      4         5     1.0
13      5         1     1.0
14      5         4     1.0
15      5         9     1.0
16      5         6     1.0
17      6         2     1.0
18      6         5     1.0
19      6        10     1.0
20      6         7     1.0
21      7         3     1.0
22      7         6     1.0
23      7        11     1.0
24      8         4     1.0
25      8        12     1.0
26      8         9     1.0
27      9         5     1.0
28      9         8     1.0
29      9        13     1.0
30      9        10     1.0
31     10         6     1.0
32     10         9     1.0
33     10        14     1.0
34     10        11 

We should now be able to use this adjacency list to run comparisons between `x` and `z`

In [106]:
# First, set up a series that maps the y values (input as self.y) to the weights table 
zseries_x = pd.Series(x, index=w.id_order)
zseries_z = pd.Series(z, index=w.id_order)

# Next, map the y values to the focal (i) values 
focal_x = zseries_x.loc[adj_list.focal].values
focal_z = zseries_z.loc[adj_list.focal].values

# Repeat the mapping but for the neighbor (j) values
neighbor_x = zseries_x.loc[adj_list.neighbor].values
neighbor_z = zseries_z.loc[adj_list.neighbor].values

Now the `y_1` and `y_2` vectors have been mapped to focal and neighbor objects (respectively `_var1` and `_var2`). 


In [114]:
BJC = (focal_x == 1) & (focal_z == 0) & (neighbor_x == 0) & (neighbor_z == 1)
BJC

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [117]:
# Create a df that uses the adjacency list focal values and the BBs counts
temp = pd.DataFrame(adj_list.focal.values, BJC.astype('uint8')).reset_index()
# Temporarily rename the columns
temp.columns = ['BJC', 'ID']
temp = temp.groupby(by='ID').sum()
temp.BJC.values

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0], dtype=uint64)

Appears to be working! Now onto the next case...

**Case 2:**